<a href="https://colab.research.google.com/github/krishnamani77/TF2_Notebooks/blob/master/Sentiment-analysis-IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
import keras.datasets

In [21]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
# IMDB Sentiment analysis
num_words_max = 3000
(X, y), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=num_words_max)

In [0]:
word_index = keras.datasets.imdb.get_word_index()
reverse_index = {}
for k, v in word_index.items():
  reverse_index[v+3] = k

In [24]:
print(len(word_index), len(reverse_index))

88584 88584


In [25]:
print(X[0])
print(y[:10])
print(y[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 2, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]
[1 0 0 1 0 0 1 0 1 0]
1


In [0]:
for id, token in enumerate(("<pad>", "<sos>", "<unk")):
  reverse_index[id] = token

In [0]:
def get_text(idx_array):
  return ' '.join([reverse_index[idx] for idx in idx_array])

In [33]:
#s = ' '.join([reverse_index[idx] for idx in X[0][:10]])
print(get_text(X[0]))

<sos> this film was just brilliant casting location scenery story direction <unk really <unk the part they played and you could just imagine being there robert <unk is an amazing actor and now the same being director <unk father came from the same <unk island as myself so i loved the fact there was a real connection with this film the witty <unk throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <unk and would recommend it to everyone to watch and the fly <unk was amazing really <unk at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <unk to the two little <unk that played the <unk of <unk and paul they were just brilliant children are often left out of the <unk list i think because the stars that play them all grown up are such a big <unk for the whole film but these children are amazing and should be <unk for what they have done don't you think the

In [0]:
vocab_size = len(word_index)

In [0]:
import numpy as np
def vectorize(sequences, dimension = num_words_max):
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1
  return results

X = vectorize(X, num_words_max)
y = np.array(y).astype("float32")
X_test = vectorize(X_test, num_words_max)
y_test = np.array(y_test).astype("float32")

In [11]:
print(X.shape, y.shape)
spl = int(0.8 * len(X))
print(spl)
X_train = X[:spl]
X_val = X[spl:]
y_train = y[:spl]
y_val = y[spl:]

(25000, 3000) (25000,)
20000


In [12]:
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(20000, 3000) (20000,) (5000, 3000) (5000,)


In [0]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense

def get_rnn_model(vocab_size, embed_size, input_shape):
  embedding = Embedding(vocab_size, embed_size, input_shape=(num_words_max, ))
  hidden1 = GRU(128, return_sequences = True)
  hidden2 = GRU(128)
  dense = Dense(1, activation='sigmoid')
  model = Sequential([embedding, hidden1, hidden2, dense])
  return model

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

def get_simple_model(input_shape):
  model = Sequential()
  # Input - Layer
  model.add(Dense(50, activation = "relu", input_shape=input_shape))
  # Hidden - Layers
  model.add(Dropout(0.3, noise_shape=None, seed=None))
  model.add(Dense(50, activation = "relu"))
  model.add(Dropout(0.2, noise_shape=None, seed=None))
  model.add(Dense(50, activation = "relu"))
  # Output- Layer
  model.add(Dense(1, activation = "sigmoid"))
  return model

In [15]:
model = get_simple_model(input_shape=(num_words_max, ))
embed_size = 128
#model = get_rnn_model(vocab_size, embed_size, (num_words_max, ))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=128, epochs=5)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                150050    
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 155,201
Trainable params: 155,201
Non-trainable params: 0
________________________________________________

In [0]:
#for idx_array,actual in zip(X_test[:5], y_test[:5]):
  #print(get_text(idx_array))
#  print('The predicted sentiment is {} and the actual sentiment is {} '.format(model.predict(vectorize(idx_array)), actual))

In [47]:
print(np.mean(history.history["val_accuracy"]))

0.8777200102806091
